In [3]:
%tensorflow_version 2.x

In [2]:
!pip install seqeval transformers==3.1.0

     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 884 kB 9.3 MB/s 
     |████████████████████████████████| 1.2 MB 45.8 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 3.0 MB 44.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=c65109d4c28188d18d15ce999f15f4edbe775d14f6fc72f3a456e288c0fff0d9
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [4]:
!mkdir data
!mkdir models
!wget https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll -P data/

--2021-12-20 04:46:02--  https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297592 (1.2M) [text/plain]
Saving to: ‘data/ja.wikipedia.conll’

ja.wikipedia.conll  100%[===================>]   1.24M  --.-KB/s    in 0.05s   

2021-12-20 04:46:02 (23.6 MB/s) - ‘data/ja.wikipedia.conll’ saved [1297592/1297592]



In [5]:
import re

import numpy as np
import tensorflow as tf
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
batch_size = 32
epochs = 100
num_words = 15000

### データセットの準備

In [11]:
def load_dataset(filename, encoding='utf-8'):
    sents, labels = [], []
    words, tags = [], []
    with open(filename, encoding=encoding) as f:
        for line in f:
            line = line.rstrip() #末尾の空白を削除
            if line: #空文字列かを判別
                word, tag = line.split('\t')
                words.append(word)
                tags.append(tag)
            else:
                sents.append(words)
                labels.append(tags)
                words, tags = [], []
    return sents, labels

In [12]:
x, y = load_dataset('./data/ja.wikipedia.conll')

### 前処理

In [13]:
class Vocab:

    def __init__(self, num_words=None, lower=True, oov_token=None):
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
            num_words=num_words,
            oov_token=oov_token,
            filters='',
            lower=lower,
            split='\t'
        )

    def fit(self, sequences):
        texts = self._texts(sequences)
        self.tokenizer.fit_on_texts(texts)
        return self

    #テキストデータ→数列データへの変換
    def encode(self, sequences):
        texts = self._texts(sequences)
        return self.tokenizer.texts_to_sequences(texts)
    #数列データ→テキストデータへの変換
    def decode(self, sequences):
        texts = self.tokenizer.sequences_to_texts(sequences)
        return [text.split(' ') for text in texts]

    #タブ区切りでtextを結合
    def _texts(self, sequences):
        return ['\t'.join(words) for words in sequences]

    #インデックス番号の確認
    def get_index(self, word):
        return self.tokenizer.word_index.get(word)

    @property
    def size(self):
        """Return vocabulary size."""
        return len(self.tokenizer.word_index) + 1

    def save(self, file_path):
        with open(file_path, 'w') as f:
            config = self.tokenizer.to_json()
            f.write(config)

    #classを呼び出さなくても使用可能
    @classmethod
    def load(cls, file_path):
        with open(file_path) as f:
            tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
            vocab = cls()
            vocab.tokenizer = tokenizer
        return vocab

#数字を0に統一
def normalize_number(text, reduce=True):
    if reduce:
        normalized_text = re.sub(r'\d+', '0', text)
    else:
        normalized_text = re.sub(r'\d', '0', text)
    return normalized_text

def preprocess_dataset(sequences):
    sequences = [[normalize_number(w) for w in words] for words in sequences]
    return sequences

def create_dataset(sequences, vocab):
    sequences = vocab.encode(sequences)
    sequences = pad_sequences(sequences, padding='post')
    return sequences

In [12]:
x = preprocess_dataset(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
source_vocab = Vocab(num_words=num_words, oov_token='<UNK>').fit(x_train)
target_vocab = Vocab(lower=False).fit(y_train)
x_train = create_dataset(x_train, source_vocab)
y_train = create_dataset(y_train, target_vocab)

### モデルの構築

In [13]:
class UnidirectionalModel:

    def __init__(self, input_dim, output_dim, emb_dim=100, hid_dim=100, embeddings=None):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       weights=[embeddings],
                                       name='embedding')
        self.lstm = LSTM(hid_dim,
                         return_sequences=True, #
                         name='lstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        lstm = self.lstm(embedding)
        y = self.fc(lstm)
        return Model(inputs=x, outputs=y)


class BidirectionalModel:

    def __init__(self, input_dim, output_dim, emb_dim=100, hid_dim=100, embeddings=None):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       weights=[embeddings],
                                       name='embedding')
        lstm = LSTM(hid_dim,
                    return_sequences=True,
                    name='lstm')
        self.bilstm = Bidirectional(lstm, name='bilstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        lstm = self.bilstm(embedding)
        y = self.fc(lstm)
        return Model(inputs=x, outputs=y)

In [14]:
model = UnidirectionalModel(num_words, target_vocab.size).build()

### 学習

In [15]:
model_path = 'models/inidirectional_model.h5'
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Preparing callbacks.
callbacks = [
    EarlyStopping(patience=3),
    ModelCheckpoint(model_path, save_best_only=True)
]

# Train the model.
model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=callbacks,
          shuffle=True)

Epoch 1/100
23/23 [==============================] - 26s 726ms/step - loss: 1.3792 - val_loss: 0.7192
Epoch 2/100
23/23 [==============================] - 9s 364ms/step - loss: 0.6108 - val_loss: 0.6233
Epoch 3/100
23/23 [==============================] - 9s 376ms/step - loss: 0.5745 - val_loss: 0.6026
Epoch 4/100
23/23 [==============================] - 13s 554ms/step - loss: 0.5471 - val_loss: 0.5672
Epoch 5/100
23/23 [==============================] - 10s 426ms/step - loss: 0.5008 - val_loss: 0.5138
Epoch 6/100
23/23 [==============================] - 6s 266ms/step - loss: 0.4393 - val_loss: 0.4602
Epoch 7/100
23/23 [==============================] - 6s 266ms/step - loss: 0.3836 - val_loss: 0.4229
Epoch 8/100
23/23 [==============================] - 6s 264ms/step - loss: 0.3451 - val_loss: 0.3988
Epoch 9/100
23/23 [==============================] - 6s 262ms/step - loss: 0.3175 - val_loss: 0.3802
Epoch 10/100
23/23 [==============================] - 6s 266ms/step - loss: 0.2946 - val

### 評価

In [29]:
class InferenceAPI:

    def __init__(self, model, source_vocab, target_vocab):
        self.model = model
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def predict_from_sequences(self, sequences):
        lengths = map(len, sequences) #sequencesの長さのリスト作成
        sequences = self.source_vocab.encode(sequences) #パディング
        sequences = pad_sequences(sequences, padding='post')
        y_pred = self.model.predict(sequences)
        print(y_pred[1])
        y_pred = np.argmax(y_pred, axis=-1) #確率値の最も高いラベルのIDを取得
        print(y_pred[1])
        y_pred = self.target_vocab.decode(y_pred) #デコードして、文字列を取得
        print(y_pred[1])
        y_pred = [y[:l] for y, l in zip(y_pred, lengths)]
        return y_pred

In [30]:
model_names = ['Unidirectional Model', 'Bidirectional Model']
model = load_model(model_path)
api = InferenceAPI(model, source_vocab, target_vocab)
y_pred = api.predict_from_sequences(x_test)
print(classification_report(y_test, y_pred, digits=4))

[[4.06186692e-02 4.57269214e-02 4.21354324e-02 ... 4.04713899e-02
  4.08696048e-02 3.99204455e-02]
 [3.77860735e-03 7.61342585e-01 4.86813299e-03 ... 4.93240822e-03
  6.34047762e-03 3.80113861e-03]
 [1.50670871e-06 9.98386145e-01 1.37898005e-05 ... 5.71307282e-06
  8.44182887e-06 3.07779487e-06]
 ...
 [2.98709431e-08 9.99532938e-01 4.54658175e-06 ... 5.14279407e-07
  3.88201414e-07 1.46793582e-07]
 [2.98709431e-08 9.99532938e-01 4.54658175e-06 ... 5.14279407e-07
  3.88201414e-07 1.46793582e-07]
 [2.98709431e-08 9.99532938e-01 4.54658175e-06 ... 5.14279407e-07
  3.88201414e-07 1.46793582e-07]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]
['O',

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    ARTIFACT     0.0403    0.0325    0.0360       154
        DATE     0.4228    0.8254    0.5591       315
       EVENT     0.0000    0.0000    0.0000        64
    LOCATION     0.6410    0.5057    0.5654       526
       MONEY     0.0000    0.0000    0.0000        12
      NUMBER     0.0781    0.1147    0.0929       218
ORGANIZATION     0.1292    0.1855    0.1523       248
       OTHER     0.0000    0.0000    0.0000        75
     PERCENT     0.0000    0.0000    0.0000        52
      PERSON     0.1681    0.0893    0.1166       224
        TIME     0.0000    0.0000    0.0000         5

   micro avg     0.3188    0.3286    0.3236      1893
   macro avg     0.1345    0.1594    0.1384      1893
weighted avg     0.2975    0.3286    0.2975      1893



### 双方向LSTM

In [31]:
class BidirectionalModel:

    def __init__(self, input_dim, output_dim, emb_dim=100, hid_dim=100, embeddings=None):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       weights=[embeddings],
                                       name='embedding')
        lstm = LSTM(hid_dim,
                    return_sequences=True,
                    name='lstm')
        self.bilstm = Bidirectional(lstm, name='bilstm')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        lstm = self.bilstm(embedding)
        y = self.fc(lstm)
        return Model(inputs=x, outputs=y)

In [32]:
models = BidirectionalModel(num_words, target_vocab.size).build()

In [36]:
model_path = 'models/bidirectional_model.h5'
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Preparing callbacks.
callbacks = [
    EarlyStopping(patience=3),
    ModelCheckpoint(model_path, save_best_only=True)
]

# Train the model.
model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          callbacks=callbacks,
          shuffle=True)

Epoch 1/100
23/23 [==============================] - 12s 325ms/step - loss: 0.0901 - val_loss: 0.2921
Epoch 2/100
23/23 [==============================] - 6s 263ms/step - loss: 0.0825 - val_loss: 0.2918
Epoch 3/100
23/23 [==============================] - 6s 260ms/step - loss: 0.0758 - val_loss: 0.2952
Epoch 4/100
23/23 [==============================] - 6s 262ms/step - loss: 0.0697 - val_loss: 0.3029
Epoch 5/100
23/23 [==============================] - 6s 262ms/step - loss: 0.0641 - val_loss: 0.3059


In [37]:
model_names = ['Unidirectional Model', 'Bidirectional Model']
model = load_model(model_path)
api = InferenceAPI(model, source_vocab, target_vocab)
y_pred = api.predict_from_sequences(x_test)
print(classification_report(y_test, y_pred, digits=4))

[[2.6709259e-02 5.3653013e-02 4.8843402e-02 ... 3.5443041e-02
  3.7477136e-02 3.7296943e-02]
 [6.1948107e-05 9.5923787e-01 1.6383578e-04 ... 4.1339503e-04
  1.0973122e-03 5.3286948e-04]
 [8.4591489e-10 9.9984336e-01 1.1000650e-07 ... 2.5120201e-07
  6.3657632e-07 3.9887053e-07]
 ...
 [1.7230545e-11 9.9993765e-01 7.4459898e-08 ... 5.5386899e-08
  6.0474413e-08 8.4688679e-08]
 [1.7230545e-11 9.9993765e-01 7.4459898e-08 ... 5.5386899e-08
  6.0474413e-08 8.4688679e-08]
 [1.7230545e-11 9.9993765e-01 7.4459898e-08 ... 5.5386899e-08
  6.0474413e-08 8.4688679e-08]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    ARTIFACT     0.1486    0.1429    0.1457       154
        DATE     0.5355    0.6698    0.5952       315
       EVENT     0.0217    0.0156    0.0182        64
    LOCATION     0.6404    0.5247    0.5768       526
       MONEY     0.0000    0.0000    0.0000        12
      NUMBER     0.2652    0.5000    0.3466       218
ORGANIZATION     0.1629    0.2016    0.1802       248
       OTHER     0.0244    0.0133    0.0172        75
     PERCENT     0.0000    0.0000    0.0000        52
      PERSON     0.2743    0.1384    0.1840       224
        TIME     0.0000    0.0000    0.0000         5

   micro avg     0.3599    0.3703    0.3650      1893
   macro avg     0.1885    0.2006    0.1876      1893
weighted avg     0.3652    0.3703    0.3578      1893



### BERT

In [7]:
def evaluate(model, target_vocab, features, labels):
    label_ids = model.predict(features)[0]
    label_ids = np.argmax(label_ids, axis=-1)
    y_pred = [[] for _ in range(label_ids.shape[0])]
    y_true = [[] for _ in range(label_ids.shape[0])]
    for i in range(label_ids.shape[0]):
        for j in range(label_ids.shape[1]):
            if labels[i][j] == 0:
                continue
            y_pred[i].append(label_ids[i][j])
            y_true[i].append(labels[i][j])
    y_pred = target_vocab.decode(y_pred)
    y_true = target_vocab.decode(y_true)
    print(classification_report(y_true, y_pred, digits=4))

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


def convert_examples_to_features(x, y,
                                 vocab,
                                 max_seq_length,
                                 tokenizer):
    pad_token = 0
    features = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': [],
        'label_ids': []
    }
    for words, labels in zip(x, y):
        tokens = [tokenizer.cls_token]
        label_ids = [pad_token]
        for word, label in zip(words, labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            label_id = vocab.get_index(label)
            label_ids.extend([label_id] + [pad_token] * (len(word_tokens) - 1))
        tokens += [tokenizer.sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)
        token_type_ids = [pad_token] * max_seq_length

        features['input_ids'].append(input_ids)
        features['attention_mask'].append(attention_mask)
        features['token_type_ids'].append(token_type_ids)
        features['label_ids'].append(label_ids)

    for name in features:
        features[name] = pad_sequences(features[name], padding='post', maxlen=max_seq_length)

    x = [features['input_ids'], features['attention_mask'], features['token_type_ids']]
    y = features['label_ids']
    return x, y

In [9]:
import tensorflow as tf
from transformers import TFBertForTokenClassification, BertConfig


def build_model(pretrained_model_name_or_path, num_labels):
    config = BertConfig.from_pretrained(
        pretrained_model_name_or_path,
        num_labels=num_labels
    )
    model = TFBertForTokenClassification.from_pretrained(
        pretrained_model_name_or_path,
        config=config
    )
    model.layers[-1].activation = tf.keras.activations.softmax
    return model


def loss_func(num_labels):
    loss_fct = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

    def loss(y_true, y_pred):
        input_mask = tf.not_equal(y_true, 0)
        logits = tf.reshape(y_pred, (-1, num_labels))
        active_loss = tf.reshape(input_mask, (-1,))
        active_logits = tf.boolean_mask(logits, active_loss)
        train_labels = tf.reshape(y_true, (-1,))
        active_labels = tf.boolean_mask(train_labels, active_loss)
        cross_entropy = loss_fct(active_labels, active_logits)
        return cross_entropy
    return loss

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertJapaneseTokenizer


def main():
    # Set hyper-parameters.
    batch_size = 16
    epochs = 100
    model_path = 'models/'
    pretrained_model_name_or_path = 'cl-tohoku/bert-base-japanese-whole-word-masking'
    maxlen = 250

    # Data loading.
    x, y = load_dataset('./data/ja.wikipedia.conll')
    tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name_or_path, do_word_tokenize=False)

    # Pre-processing.
    x = preprocess_dataset(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    target_vocab = Vocab(lower=False).fit(y_train)
    features_train, labels_train = convert_examples_to_features(
        x_train,
        y_train,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )
    features_test, labels_test = convert_examples_to_features(
        x_test,
        y_test,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )

    # Build model.
    model = build_model(pretrained_model_name_or_path, target_vocab.size)
    model.compile(optimizer='sgd', loss=loss_func(target_vocab.size))

    # Preparing callbacks.
    callbacks = [
        EarlyStopping(patience=3),
    ]

    # Train the model.
    model.fit(x=features_train,
              y=labels_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              callbacks=callbacks,
              shuffle=True)
    model.save_pretrained(model_path)

    # Evaluation.
    evaluate(model, target_vocab, features_test, labels_test)


if __name__ == '__main__':
    main()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100
45/45 [==============================] - 2017s 45s/step - loss: 0.8568 - val_loss: 0.6781
Epoch 2/100
36/45 [=======================>......] - ETA: 6:23 - loss: 0.6172